In [1]:
import pandas as pd
from datetime import timedelta, date
import geopandas as gpd
from uuid import uuid4

Things to think about:
* what if control day not in temperature data set? (solution is to add more temperature data)
* what if control day is actually a heat wave?
* what if control day is close to a heat wave?

In [2]:

from datetime import datetime, timedelta
import pandas as pd
from shapely.geometry import Point
import geopandas as gpd
#import contextily as ctx
import matplotlib.pyplot as plt
datelist = pd.date_range(datetime.today(), periods=100).tolist()
from time import sleep
import dask.dataframe as dd
from dateutil.parser import parse
from dateutil.relativedelta import relativedelta
import os
import glob

from shapely.geometry import Polygon, MultiPolygon, LineString, GeometryCollection

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import geopandas as gpd



## Load heatwaves and get initial control dates

In [3]:
#heatwaves = pd.read_csv(f'/Users/shivyucel/Documents/projects/DPhil/Code_Data/data/heatwave/heatwaves_90.csv')
heatwaves = pd.read_csv('/Users/shivyucel/Documents/projects/DPhil/Code_Data/data/heatwave/united_states/THI_united_states_90_2022.csv')

In [4]:
heatwaves['month'].unique()

array([6, 7, 8, 9, 5])

In [5]:
heatwaves['datetime'] = pd.to_datetime(heatwaves['datetime'])

In [6]:
heatwaves['control_day'] = heatwaves['datetime'] - timedelta(days=7)


In [7]:
heatwaves['day_of_week'] = [x.weekday() for x in heatwaves['datetime']]

In [8]:
heatwaves_active = heatwaves[heatwaves['heatwave'] == 1]
heat_days = heatwaves_active.groupby('location')['datetime'].unique().reset_index().set_index('location')

In [9]:
heat_days = heat_days.to_dict()['datetime']

In [10]:
heatwaves_active.reset_index(inplace=True)

In [11]:
full_df = pd.read_csv('/Users/shivyucel/Documents/projects/DPhil/Code_Data/data/THI/united_states/daily_max_THI.csv')

full_df.rename(columns={'time': 'datetime'}, inplace=True)
full_df['datetime'] = pd.to_datetime(full_df['datetime'])
full_df['day'] = pd.DatetimeIndex(full_df['datetime']).day
full_df['month'] = pd.DatetimeIndex(full_df['datetime']).month
full_df['year'] = pd.DatetimeIndex(full_df['datetime']).year
full_df = full_df[~full_df['month'].isin([4, 10])]
full_df['location'] = full_df['adm2'] + ', ' + full_df['adm1']
full_df['datetime_ord'] = [date(x,y,z).toordinal() for x,y,z in zip(full_df['year'], full_df['month'], full_df['day'])]


In [12]:
full_df['temp_pct'] = full_df.groupby('location')['thi'].rank(pct=True)

In [13]:
heatwaves_active['num_iters'] = 'test'

In [ ]:
# Updated control day search algorithm with less index searching -- faster for large datasets

for index, row in heatwaves_active.iterrows():
    subset = full_df[full_df['location'] == row['location']]
    control_day = heatwaves_active.iloc[index, -3]
    i = 0
    try:
        while ((subset[subset['datetime'] == control_day]['temp_pct'].values[0]>= 0.8) | 
               (subset[(subset['datetime'] == control_day)]['temp_pct'].values[0]<= 0.2)):


            heatwaves_active.iloc[index, -3] = heatwaves_active.iloc[index, -3] - timedelta(days=7)
            control_day = heatwaves_active.iloc[index, -3]

    except:
        heatwaves_active.iloc[index, -3] = heatwaves_active.iloc[index, -3] + timedelta(days=7)
        control_day = heatwaves_active.iloc[index, -3]

        while ((subset[subset['datetime'] == control_day]['temp_pct'].values[0]>= 0.8) | 
               (subset[(subset['datetime'] == control_day)]['temp_pct'].values[0]<= 0.2)):
        
            heatwaves_active.iloc[index, -3] = heatwaves_active.iloc[index, -3] + timedelta(days=7)
            control_day = heatwaves_active.iloc[index, -3]
            
    print(index / len(heatwaves_active))
            


In [17]:
#export.to_csv('/Users/shivyucel/Documents/projects/DPhil/Code_Data/data/heatwave_control/heatwave_control_90_v2.csv')
heatwaves_active.to_csv('/Users/shivyucel/Documents/projects/DPhil/Code_Data/data/paper2/united_states/THI_heatwave_control_90.csv')

In [35]:
df = pd.read_csv('/Users/shivyucel/Documents/projects/DPhil/Code_Data/data/paper2/united_states/heatwave_control_90.csv')